In [ ]:
import sys

# Add parent directory to path to import shared utilities
sys.path.insert(0, '..')

# Import shared utilities
from benchmark_utils import *

# Configuration specific to this notebook
#
# Root directory where the benchmarking results are stored
root_dir = "."

# Find the folders with this prefix in the root directory, e.g.: meta-llama_Llama-3.3-70B-Instruct-openai-chat-concurrency
directory_prefix = ""

# Cost of the VM instance used for e.g.: https://instances.vantage.sh/azure/vm/nc96ads-v4 14.692
hourly_cost = 0.0

# Using 100% of the GPUs on the node then the value is 1.0, if using more than one machine then increase value accordingly.
percent_gpu_used = 1

# This prefix shows up on each plot's title.
prefix = "llmd: (prefill + decode) 4 GPUs"

# Concurrency values to use to plot the data, the default concurrencies can be used from the variable DEFAULT_CONCURRENCIES
# DEFAULT_CONCURRENCIES = [1, 2, 5, 10, 50, 100, 250]
concurrencies = [1, 2, 5, 10]

# Load data using shared function
all_data = load_all_data(
    DEFAULT_INPUT_OUTPUT,
    concurrencies,
    root_dir,
    directory_prefix,
    hourly_cost,
    percent_gpu_used
)

plot_everything(all_data, prefix)